In [1]:
from __future__ import print_function
import numpy as np
import torch
import math
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

In [9]:
batch_size = 64

device = 'cuda' if cuda.is_available() else 'cpu'

train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)
test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

train_loader = data.DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_loader = data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

In [10]:
class net(nn.Module):
    def __init__(self):
        super(net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5) #input Dimension, output D, filter size
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5)
        self.mp = nn.MaxPool2d(2) #kernel size
        self.fcl1 = nn.Linear(320, 10) #28*28 imag x kernal 5 -> 24*24 img x pool2*2 -> 12*12 img x kernal 5 -> 8*8img x pool2*2 -> 4*4 * 20dim = 320
    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)
        return self.fcl1(x)
    
    
model = net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.5)

In [11]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        output = model(data)
        loss = criterion(output, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if batch_idx % 10 == 0:
        print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [12]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim = True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

In [13]:
for epoch in range(1, 10):
        train(epoch)
        test()

Test set: Average loss: 0.0026, Accuracy: 9514/10000 (95%)
Test set: Average loss: 0.0016, Accuracy: 9695/10000 (97%)
Test set: Average loss: 0.0012, Accuracy: 9774/10000 (98%)
Test set: Average loss: 0.0011, Accuracy: 9792/10000 (98%)
Test set: Average loss: 0.0011, Accuracy: 9764/10000 (98%)
Test set: Average loss: 0.0010, Accuracy: 9796/10000 (98%)
Test set: Average loss: 0.0009, Accuracy: 9803/10000 (98%)
Test set: Average loss: 0.0009, Accuracy: 9814/10000 (98%)
Test set: Average loss: 0.0007, Accuracy: 9852/10000 (99%)
